# Uso de AutoCodificadores imagenes - reducción de dimensiones

## Los Datos

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from tensorflow.keras.datasets import mnist

In [ ]:
(X_entreno, y_entreno), (X_prueba, y_prueba) = mnist.load_data()

In [ ]:
plt.imshow(X_entreno[0])

### Se normalizan los datos (escala original 0 - 255)

In [ ]:
X_entreno = X_entreno/255
X_prueba = X_prueba/255

## AutoCodificador Básico

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Reshape
# from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers.legacy import SGD #para uso con procesadores M! y M2

Es buena idea armar lo que se llama un "AutoCodificador Apilado" en el cual cada capa va rebajando el número de neuronas hasta el número que desee. 

Esto es principalmente para que el aprendizaje pueda ser más escalonado y no intentar que sea de un solo golpe.

Para este ejercicio se irá bajando aproximadamente de mitad en mitad, hasta llegar a una capa de tan solo 25 neuronas.

Esto es totalmente arbitrario y pueden escogerse otros esquémas

In [ ]:
784/2

In [ ]:
codificador = Sequential()
codificador.add(Flatten(input_shape = [28, 28]))
codificador.add(Dense(400, activation = "relu"))
codificador.add(Dense(200, activation = "relu"))
codificador.add(Dense(100, activation = "relu"))
codificador.add(Dense(50, activation = "relu"))
codificador.add(Dense(25, activation = "relu"))

La capa final de codificador es de 25 unidades,  El decodificador toma esa como entrada y va aumentando hasta llegar a la dimensión correcta.

Como se desea comparar con la imagen original, es necesario regresar al formato original de 28 X 28

In [ ]:
decodificador = Sequential()
decodificador.add(Dense(50, input_shape = [25],
                        activation = 'relu'))
decodificador.add(Dense(100, activation = 'relu'))
decodificador.add(Dense(200, activation = 'relu'))
decodificador.add(Dense(400, activation = 'relu'))
decodificador.add(Dense(784, activation = "sigmoid"))
decodificador.add(Reshape([28, 28]))

### Conformar el autocodificador completo.

Se utiliza la función de activación *sigmoid* porque usaremos la función de pérdida *binary_crossenthropy*

Se utilza la función de pérdida *binary_crossenthropy* porque no interesa obtener clases sino conocer si la imagen resultante es similar a la original.

In [ ]:
autocodificador = Sequential([codificador, 
                              decodificador])
autocodificador.compile(loss = "binary_crossentropy", optimizer = SGD(learning_rate = 1.5), metrics = ['accuracy'])

In [ ]:
autocodificador.fit(X_entreno, X_entreno, 
                    epochs = 5,
                    validation_data = [X_prueba, 
                                       X_prueba])

Ver si el autocodificador puede predecir (en este caso, reconstruir) las imágenes que recibe

In [ ]:
imagenes_generadas = autocodificador.predict(X_prueba[:10])

In [ ]:
n = 4   # Seleccionar la imagen que se desea comparar

print("Imagen original")
plt.imshow(X_prueba[n]);
plt.show()
print("Imagen reconstruida con el autocodificador")
plt.imshow(imagenes_generadas[n]);

Para probar con otra imagen...ejecutar celda anterior con otro valor de n.

Bastante impresionante...se redujeron las dimensiones de 28 X 28 = 784 a tan solo 25, o sea solo el 3% de las dimensiones originales, y se lograron imágenes bastante parecidas!

# AutoCodificadores para reducir el ruido en imágenes

In [ ]:
from tensorflow.keras.layers import GaussianNoise

### Ver cómo funciona el generador de ruido

In [ ]:
muestra = GaussianNoise(0.2)   # Desviación estándard, si se quiere más ruido, incrementar

In [ ]:
ruidosas = muestra(X_prueba[:10], training = True)

In [ ]:
n = 0   # Seleccionar la imagen que se desea comparar

print("Imagen original")
plt.imshow(X_prueba[n]);
plt.show()
print("Imagen ruidosa")
plt.imshow(ruidosas[n]);

Ver otra...otro valor de n

### Crear autocodificador para eliminación de ruido y entrenarlo.

In [ ]:
import tensorflow as tf
import numpy as np

La siguiente celda es opcional, se usa para tener reproducibilidad

In [ ]:
tf.random.set_seed(101)
np.random.seed(101)

Se usará el mismo autocodificador, solo que con una capa más para introducir el ruido a las imágenes,  en el codificador

In [ ]:
codificador = Sequential()
codificador.add(Flatten(input_shape = [28, 28]))

# Agregar ruido a las imágenes
codificador.add(GaussianNoise(0.2))

codificador.add(Dense(400, activation = "relu"))
codificador.add(Dense(200, activation = "relu"))
codificador.add(Dense(100, activation = "relu"))
codificador.add(Dense(50, activation = "relu"))
codificador.add(Dense(25, activation = "relu"))

In [ ]:
decodificador = Sequential()
decodificador.add(Dense(50, input_shape = [25],
                        activation = 'relu'))
decodificador.add(Dense(100, activation = 'relu'))
decodificador.add(Dense(200, activation = 'relu'))
decodificador.add(Dense(400, activation = 'relu'))
decodificador.add(Dense(784, activation = "sigmoid"))
decodificador.add(Reshape([28, 28]))

### Conformar el Autocodificador

In [ ]:
eliminador_ruido = Sequential([codificador, 
                               decodificador])

In [ ]:
eliminador_ruido.compile(loss = "binary_crossentropy", 
                         optimizer = 'adam', 
                         metrics = ['accuracy'])

In [ ]:
eliminador_ruido.fit(X_entreno, X_entreno, 
                     epochs = 8,
                     validation_data = [X_prueba, X_prueba])

### Ver resultados con una muestra de las imágenes

In [ ]:
diez_imags_con_ruido = muestra(X_prueba[: 10], 
                                 training = True)
limpias = eliminador_ruido(diez_imags_con_ruido[: 10])

**OJO!!!**  Nótese que en la celda anterior, realmente se ha "duplicado" la cantidad de ruido:  una vez por generar imágenes con ruido (primera instrucción), y una segunda vez por pasarlo por el codificador! 

In [ ]:
n = 0
print("La Original")
plt.imshow(X_prueba[n])
plt.show()
print("La version con ruido")
plt.imshow(diez_imags_con_ruido[n])
plt.show()
print("Luego de pasar por el eliminidador de ruido")
plt.imshow(limpias[n])
plt.show()